In [4]:
pip install transformers

     ---------------------------------------- 7.2/7.2 MB 17.8 kB/s eta 0:00:00
     ---------------------------------------- 3.5/3.5 MB 82.3 kB/s eta 0:00:00
     ------------------------------------- 263.7/263.7 kB 79.5 kB/s eta 0:00:00
     ------------------------------------- 268.0/268.0 kB 72.4 kB/s eta 0:00:00
     ------------------------------------- 236.8/236.8 kB 48.2 kB/s eta 0:00:00
     ------------------------------------- 163.8/163.8 kB 69.2 kB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

Looking in indexes: https://download.pytorch.org/whl/cu117
                                              0.0/2.3 GB 29.2 kB/s eta 22:07:07
Note: you may need to restart the kernel to use updated packages.


ERROR: Exception:
Traceback (most recent call last):
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 435, in _error_catcher
    yield
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\urllib3\response.py", line 516, in read
    data = self._fp.read(amt) if not fp_closed else b""
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\site-packages\pip\_vendor\cachecontrol\filewrapper.py", line 90, in read
    data = self.__fp.read(amt)
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\http\client.py", line 465, in read
    s = self.fp.read(amt)
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
  File "C:\Users\India\AppData\Local\Programs\Python\Python310\lib\ssl.py", line 1274, in recv_into
    return self.read(nbytes, buffer)
  File "C:\Users\India\AppData\Lo

In [1]:
import os
import librosa
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoModelForAudioClassification, AutoTokenizer, Wav2Vec2ForCTC, Wav2Vec2Tokenizer
import IPython.display as ipd

C:\Users\India\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the paths to the datasets and translated output folder
datasets_path = "E:/AudioPalm_MODEL/Voices"
translated_output_folder = "E:/AudioPalm_MODEL/translated_english_audio"

In [3]:
# Initialize the AudioPalm LLM model
model = AutoModelForAudioClassification.from_pretrained("facebook/audiopalm-base")
tokenizer = AutoTokenizer.from_pretrained("facebook/audiopalm-base")

OSError: facebook/audiopalm-base is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo with `use_auth_token` or log in with `huggingface-cli login` and pass `use_auth_token=True`.

In [ ]:
# Initialize the Wav2Vec2 model for translation
translation_model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
translation_tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

In [ ]:
# Load the datasets
datasets = []
labels = []
for language in os.listdir(datasets_path):
    dataset_path = os.path.join(datasets_path, language)
    audio_files = os.listdir(dataset_path)
    for audio_file in audio_files:
        audio_path = os.path.join(dataset_path, audio_file)
        audio_features = librosa.feature.melspectrogram(librosa.load(audio_path))
        datasets.append(audio_features)
        labels.append(language)

In [ ]:
# Convert the labels to numeric representation
label_mapping = {label: i for i, label in enumerate(set(labels))}
labels = [label_mapping[label] for label in labels]

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    datasets, labels, test_size=0.4, random_state=42
)

# Train the AudioPalm LLM model
model.train_on_batch(X_train, y_train)


In [ ]:
# Translate and save the audio and text for the testing set
translated_output_folder_audio = os.path.join(translated_output_folder, "translated_english_audio")
translated_output_folder_text = os.path.join(translated_output_folder, "translated_english_text")
os.makedirs(translated_output_folder_audio, exist_ok=True)
os.makedirs(translated_output_folder_text, exist_ok=True)

In [ ]:
for i, audio_features in enumerate(X_test):
    # Convert audio features to wav format
    waveform = librosa.feature.inverse.mel_to_audio(audio_features)
    wav_filename = f"audio_{i}.wav"
    wav_path = os.path.join(translated_output_folder_audio, wav_filename)
    librosa.output.write_wav(wav_path, waveform, sr=22050)

    # Perform audio translation
    input_values = translation_tokenizer(wav_path, return_tensors="pt").input_values
    logits = translation_model(input_values).logits
    predicted_ids = np.argmax(logits, axis=-1)
    transcription = translation_tokenizer.decode(predicted_ids[0])
    txt_filename = f"transcription_{i}.txt"
    txt_path = os.path.join(translated_output_folder_text, txt_filename)
    with open(txt_path, "w") as txt_file:
        txt_file.write(transcription)

    # Playback of randomly selected audio samples and translations
    if i < 5:
        print("Sample", i + 1)
        print("Original audio (Hindi):")
        ipd.display(ipd.Audio(librosa.feature.inverse.mel_to_audio(X_test[i]), rate=22050))
        print("Translation (English):", transcription)
        print("Translated audio (English):")
        ipd.display(ipd.Audio(wav_path))

print("Translation completed and saved.")

In [ ]:
# Evaluate the model on the testing set
results = model.evaluate(X_test, y_test)
print("Test loss:", results[0])
print("Test accuracy:", results[1])

# Save the trained AudioPalm LLM model
model.save_pretrained("audiopalm-model")